In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img, save_img
from keras.models import Sequential,Model
from keras.layers import Conv2D, GlobalMaxPooling2D,GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Concatenate, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import SGD, Adam
from sklearn.metrics import roc_auc_score, confusion_matrix
import os
import csv
import numpy as np
import cv2
from imgaug import augmenters as iaa
import imgaug as ia

train_data_dir = "train_img"
test_dir="test_img"
img_size = (96,96)
batch_size = 64





Using TensorFlow backend.


In [2]:
def get_seq():
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.Flipud(0.2), # vertically flip 20% of all images
            sometimes(iaa.Affine(
                scale={"x": (1., 1.1), "y": (1., 1.1)}, # scale images to 80-120% of their size, individually per axis
#                 translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
                rotate=(-10, 10), # rotate by -45 to +45 degrees
                shear=(-10, 10), # shear by -16 to +16 degrees
                order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
#                 cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                mode='reflect' # use any of scikit-image's warping modes
            )),
            # execute 0 to 3 of the following (less important) augmenters per image
            # don't execute all of them, as that would often be way too strong
            iaa.SomeOf((0, 3),
                [
#                     sometimes(iaa.Superpixels(p_replace=(0, .20), n_segments=(20, 200))), # convert images into their superpixel representation
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 1.0)), # blur images with a sigma between 0 and 3.0
                        iaa.AverageBlur(k=(3, 5)), # blur image using local means with kernel sizes between 2 and 7
                        iaa.MedianBlur(k=(3, 5)), # blur image using local medians with kernel sizes between 2 and 7
                    ]),
#                     iaa.Sharpen(alpha=(0, .5), lightness=(0.9, 1.1)), # sharpen images
#                     iaa.Emboss(alpha=(0, .5), strength=(0, 1.5)), # emboss images
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.5), # add gaussian noise to images
#                     iaa.OneOf([
#                         iaa.Dropout((0.01, 0.05), per_channel=0.5), # randomly remove up to 10% of the pixels
#                         iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),
#                     ]),
#                     iaa.Invert(0.01, per_channel=True), # invert color channels
                    iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                    # change the brightness of the whole image (sometimes
                    # per channel)
                    iaa.Multiply((0.9, 1.1), per_channel=0.5),
#                     sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
#                     sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
#                     sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                ],
                random_order=True
            )
        ],
        random_order=True
    )
    return seq

In [3]:
# Set up image generators for data augmentation
def my_gen(generator,is_augment=False):
    while True:
        x_batch,y_batch = next(generator)
#         x_batch_view = x_batch.view('uint8')
#         x_batch_view[:] = x_batch
        x_batch = x_batch.astype('uint8',copy=False)
        if is_augment:
            x_batch = seq.augment_images(x_batch)
        x_batch = preprocess_input(x_batch)
        yield x_batch,y_batch


In [68]:
seq=get_seq()

datagen = ImageDataGenerator(
    validation_split=.2,
    dtype=np.uint8
)



train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data
train_steps = len(train_generator)
train_generator = my_gen(train_generator,is_augment=True)
validation_generator = datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data
validation_steps = len(validation_generator)
validation_generator = my_gen(validation_generator)


Found 176021 images belonging to 2 classes.
Found 44004 images belonging to 2 classes.


In [5]:
def train(model,callbacks=None,epochs=3,is_save=False):
    model.fit_generator(
        train_generator,
        steps_per_epoch=train_steps, 
        epochs=epochs,
        verbose=1,
        validation_data=validation_generator, 
        validation_steps=validation_steps,
        callbacks=callbacks,
        use_multiprocessing=False
    )
    
    if is_save:
        model.save_weights('meta_model.h5')
    

In [6]:
# For architecture, refer to below link
# https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202
def inception_v2(inputs,n_filters=6,kernel_size=(3,3),pool_size=(2,2),padding="same"):
    tower_0 = Conv2D(n_filters, (1,1), activation='relu', padding=padding)(inputs)
    
    tower_1 = MaxPooling2D(pool_size = pool_size,strides=1,padding=padding)(inputs)
    tower_1 = Conv2D(n_filters, (1,1), activation='relu', padding=padding)(tower_1)
    
    tower_2 = Conv2D(n_filters, (1,1), activation='relu', padding=padding)(inputs)
    tower_2_0 = Conv2D(n_filters, (3,1), activation='relu', padding=padding)(tower_2)
    tower_2_1 = Conv2D(n_filters, (1,3), activation='relu', padding=padding)(tower_2)
    
    tower_3 = Conv2D(n_filters, (1,1), activation='relu', padding=padding)(inputs)
    tower_3 = Conv2D(n_filters, (3,3), activation='relu', padding=padding)(tower_3)
    tower_3_0 = Conv2D(n_filters, (3,1), activation='relu', padding=padding)(tower_3)
    tower_3_1 = Conv2D(n_filters, (1,3), activation='relu', padding=padding)(tower_3)
    
    x = Concatenate(axis=3)([tower_0, tower_1, tower_2_0,tower_2_1,tower_3_0,tower_3_1])
    return x

In [7]:
# create the base pre-trained model
# base_model = VGG16(weights='imagenet', include_top=False,input_shape=(36,36,3))
base_model = VGG16(weights='imagenet', include_top=False,input_shape=(96,96,3))

# Inception Block
block0 = inception_v2(base_model.output)
block0 = BatchNormalization()(block0)

# add a global spatial average pooling layer
pred0 = GlobalAveragePooling2D()(block0)
pred1 = GlobalMaxPooling2D()(block0)
pred2 = Flatten()(block0)
pred = Concatenate()([pred0,pred1,pred2])
pred = Dropout(.5)(pred)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
pred = Dense(1, activation='sigmoid')(pred)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=pred)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
# for layer in base_model.layers:
#     layer.trainable = False
for layer in model.layers[:1]:
    layer.trainable = False
for layer in model.layers[1:]:
    layer.trainable = True

# Compile AFTER freezing layer
model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.0001),
              metrics=['accuracy'])

In [80]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [81]:
train(model,callbacks = callbacks_list,epochs=9,is_save=True)

Epoch 1/9
2751/2751 [==============================] - 686s 249ms/step - loss: 0.3107 - acc: 0.8732 - val_loss: 0.2874 - val_acc: 0.8796

Epoch 00001: val_acc improved from -inf to 0.87960, saving model to best_model.h5
Epoch 2/9
2751/2751 [==============================] - 540s 196ms/step - loss: 0.2224 - acc: 0.9139 - val_loss: 0.2018 - val_acc: 0.9276

Epoch 00002: val_acc improved from 0.87960 to 0.92764, saving model to best_model.h5
Epoch 3/9
2751/2751 [==============================] - 540s 196ms/step - loss: 0.1971 - acc: 0.9256 - val_loss: 0.1893 - val_acc: 0.9276

Epoch 00003: val_acc improved from 0.92764 to 0.92764, saving model to best_model.h5
Epoch 4/9
2751/2751 [==============================] - 538s 196ms/step - loss: 0.1771 - acc: 0.9339 - val_loss: 0.1328 - val_acc: 0.9532

Epoch 00004: val_acc improved from 0.92764 to 0.95323, saving model to best_model.h5
Epoch 5/9
2751/2751 [==============================] - 538s 195ms/step - loss: 0.1650 - acc: 0.9388 - val_loss:

In [77]:
# we chose to train the top block, i.e. we will freeze
# the first 4 blocks and unfreeze the rest:
for layer in model.layers[:1]:
    layer.trainable = False
for layer in model.layers[1:]:
    layer.trainable = True

model.compile(loss='binary_crossentropy',
          optimizer=SGD(lr=.001, momentum=.01,nesterov=True,clipnorm=1.0),
          metrics=['accuracy'])

In [78]:
train(model,callbacks = callbacks_list, epochs=5,is_save=True) # Save model every epoch, re-reun def train cell

Epoch 1/5
2751/2751 [==============================] - 522s 190ms/step - loss: 0.1554 - acc: 0.9451 - val_loss: 0.1195 - val_acc: 0.9594

Epoch 00001: val_acc improved from 0.94910 to 0.95944, saving model to best_model.h5
Epoch 2/5
2751/2751 [==============================] - 524s 190ms/step - loss: 0.1424 - acc: 0.9497 - val_loss: 0.1174 - val_acc: 0.9613

Epoch 00002: val_acc improved from 0.95944 to 0.96125, saving model to best_model.h5
Epoch 3/5
2751/2751 [==============================] - 525s 191ms/step - loss: 0.1412 - acc: 0.9502 - val_loss: 0.1132 - val_acc: 0.9626

Epoch 00003: val_acc improved from 0.96125 to 0.96264, saving model to best_model.h5
Epoch 4/5
2751/2751 [==============================] - 649s 236ms/step - loss: 0.1389 - acc: 0.9505 - val_loss: 0.1130 - val_acc: 0.9623

Epoch 00004: val_acc did not improve from 0.96264
Epoch 5/5
2751/2751 [==============================] - 522s 190ms/step - loss: 0.1370 - acc: 0.9514 - val_loss: 0.1124 - val_acc: 0.9628

Epoch

In [11]:
# Performs test time augmentation and outputs prediction on given model
def predict(model,x_batch):
    return (
        model.predict(x_batch) + 
        model.predict(x_batch[:,::-1,:,:]) + 
        model.predict(x_batch[:,:,::-1,:]) + 
        model.predict(x_batch[:,::-1,::-1,:])
    )/4.

In [64]:
score=0
batches=0
conf_mat = np.empty((2,2),dtype=int)
incorrect_preds = []
i=0
for x_batch,y_batch in validation_generator:
    y_pred = predict(model,x_batch)
    y_pred = y_pred.reshape(y_pred.size)
    conf_mat += confusion_matrix(y_batch,y_pred>.5)
    wrong_idxs = np.nonzero((y_pred>.5 )!= y_batch)
    
    for label,pred,img in zip(y_batch[wrong_idxs].tolist(),y_pred[wrong_idxs].tolist(),x_batch[wrong_idxs].tolist()):
        incorrect_preds.append((int(label),pred))
        img = array_to_img(img)
        save_img("preview/%s/%d.tif"%(int(label),i),x=img)
        i+=1
        
    score += roc_auc_score(y_batch,y_pred)/validation_steps
    batches += 1
    
    print("Val Batch %d/%d\t\t " %(batches,validation_steps),end="\r")
    if batches >= validation_steps:
        break

# AUC
print("AUC score = ", score)

# Confusion Matrix
# tn, fp
# fn, tp
print("Total Negatives in data: ",130908, " %.2f%%"%(130908/(130908+89117)*100))
print("Total Positives in data: ",89117, " %.2f%%"%(89117/(130908+89117)*100))
print("Confusion Matrix\n", conf_mat)
print(89117/130908.,471/839.)
print(np.sum(conf_mat[:,0])/np.sum(conf_mat[:,1]))

with open('analysis.csv', mode='w+',newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['label','pred'])
    for label,pred in incorrect_preds:
        writer.writerow((label,pred))

AUC score =  0.9941507783021482
Total Negatives in data:  130908  59.50%
Total Positives in data:  89117  40.50%
Confusion Matrix
 [[25688   469]
 [  882 17028]]
0.680760534115562 0.5613825983313468
1.5185460364633938


In [56]:
with open('analysis.csv', mode='w+',newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['label','pred'])
    for label,pred in incorrect_preds:
        writer.writerow((label,pred))

In [9]:
# model.save_weights('modelv411.h5')
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name)

model.load_weights('fullmodelv411.h5')
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 96, 96, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 96, 96, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 48, 48, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [12]:

# Predict

images = []
img_names = []
y_pred = []

batch_count=0
for img in os.listdir(test_dir):
    img_names.append(img[:-4])
    img = load_img("%s/%s"%(test_dir,img), target_size=img_size)
    img_arr = np.array(img)
    images.append(img_arr)
    img.close()
    batch_count+=1
    if batch_count>batch_size:
        batch_count=0        
        images = preprocess_input(np.array(images,dtype=float))
        y_pred.extend((predict(model,images)).tolist())
        images=[]
else:
    images = preprocess_input(np.array(images,dtype=float))
#     images=datagen.standardize(np.array(images,dtype=float))
    y_pred.extend(model.predict(images).tolist())
    images=[]

with open('submission.csv', mode='w+',newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['id','label'])
    for idx,img_name in enumerate(img_names):
        writer.writerow([img_name,y_pred[idx][0]])
        